In [5]:
from web3 import Web3
from solcx import compile_source

# Connect to a local Ethereum node (e.g., Ganache)
web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))  
# Set the default account to the first one
web3.eth.default_account = web3.eth.accounts[0]

# Solidity source code: a simple storage contract
contract_source_code = '''
pragma solidity ^0.8.0;

contract SimpleStorage {
    uint256 storedNumber;

    // Set the number
    function set(uint256 num) public {
        storedNumber = num;
    }

    // Get the number
    function get() public view returns (uint256) {
        return storedNumber;
    }
}
'''

# Compile the contract
compiled_sol = compile_source(contract_source_code)
contract_id, contract_interface = compiled_sol.popitem()

# Get the compiled bytecode and ABI
bytecode = contract_interface['bin']
abi = contract_interface['abi']

# Deploy the contract
SimpleStorage = web3.eth.contract(abi=abi, bytecode=bytecode)

# Send a transaction to deploy the contract
print("Deploying contract... Please wait.")
tx_hash = SimpleStorage.constructor().transact()

# Wait for the transaction receipt
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

# Get the deployed contract address and instance
contract_address = tx_receipt.contractAddress
simple_storage = web3.eth.contract(address=contract_address, abi=abi)

print(f"Contract deployed at address: {contract_address}")
print(f"Transaction hash: {tx_hash.hex()}")
print(f"Transaction receipt: {tx_receipt}")

# Interact with the contract - set a number
print("\nSetting the number to 42...")
tx_hash = simple_storage.functions.set(42).transact()

# Wait for the receipt of the set transaction
tx_receipt_set = web3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Set transaction receipt: {tx_receipt_set}")
print(f"Set transaction hash: {tx_hash.hex()}")

# Interact with the contract - get the stored number
stored_number = simple_storage.functions.get().call()
print(f"\nStored number: {stored_number}")

# Additional information:
# - Get the latest block number
block_number = web3.eth.get_block('latest')['number']
print(f"Current block number: {block_number}")

# - Get the balance of the default account, to see if deploying has consumed any ether
account_balance = web3.eth.get_balance(web3.eth.default_account)
print(f"Default account balance after deployment: {web3.from_wei(account_balance, 'ether')} ETH")

Deploying contract... Please wait.
Contract deployed at address: 0x53cB922d4D1B464B57aBf2ab1113005D70eD124C
Transaction hash: a3cc4823bbd4ea12b6984995dc17e3aa3149adf5ff4196f564e627082fc2b9c1
Transaction receipt: AttributeDict({'transactionHash': HexBytes('0xa3cc4823bbd4ea12b6984995dc17e3aa3149adf5ff4196f564e627082fc2b9c1'), 'transactionIndex': 0, 'blockNumber': 16, 'blockHash': HexBytes('0x559a1037b0fe8591c2c18daa1868ef3f4521825c69b2f24c852a131f9f1b71ca'), 'from': '0xa603A5Aae135c331e335201f6028F36b81DBcb0f', 'to': None, 'cumulativeGasUsed': 118807, 'gasUsed': 118807, 'contractAddress': '0x53cB922d4D1B464B57aBf2ab1113005D70eD124C', 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000